In [15]:
# %load imdb_data_to_graph.py
import numpy as np
import dgl
import pickle
import os
from sklearn.preprocessing import MultiLabelBinarizer
import torch
import sys
sys.path.append('../panrep/') 
import load_data
import argparse
import copy
import itertools

In [16]:
    parser = argparse.ArgumentParser(description='PanRep')
    parser.add_argument("--dropout", type=float, default=0.2,
            help="dropout probability")
    parser.add_argument("--n-hidden", type=int, default=60,
            help="number of hidden units") # use 16, 2 for debug
    parser.add_argument("--gpu", type=int, default=0,
            help="gpu")
    parser.add_argument("--lr", type=float, default=1e-2,
            help="learning rate")
    parser.add_argument("--n-bases", type=int, default=20,
            help="number of filter weight matrices, default: -1 [use all]")
    parser.add_argument("--n-layers", type=int, default=3,
            help="number of propagation rounds")
    parser.add_argument("-e", "--n-epochs", type=int, default=50,
            help="number of training epochs for decoder")
    parser.add_argument("-ec", "--n-cepochs", type=int, default=400,
                        help="number of training epochs for classification")
    parser.add_argument("-num_masked", "--n-masked-nodes", type=int, default=100,
                        help="number of masked nodes")
    parser.add_argument("-pct_masked_links", "--pct-masked-links", type=int, default=0.5,
                        help="number of masked links")
    parser.add_argument("-negative_rate", "--negative-rate", type=int, default=4,
                        help="number of negative examples per masked link")


    parser.add_argument("-d", "--dataset", type=str, required=True,
            help="dataset to use")
    parser.add_argument("-en", "--encoder", type=str, required=True,
                        help="Encoder to use")
    parser.add_argument("--l2norm", type=float, default=0.0000,
            help="l2 norm coef")
    parser.add_argument("--relabel", default=False, action='store_true',
            help="remove untouched nodes and relabel")
    parser.add_argument("--use-self-loop", default=False, action='store_true',
            help="include self feature as a special relation")
    parser.add_argument("--use-infomax-loss", default=False, action='store_true',
                        help="use infomax task supervision")
    parser.add_argument("--use-reconstruction-loss", default=True, action='store_true',
                        help="use feature reconstruction task supervision")
    parser.add_argument("--node-masking", default=False, action='store_true',
                        help="mask a subset of node features")
    parser.add_argument("--loss-over-all-nodes", default=True, action='store_true',
                        help="compute the feature reconstruction loss over all nods or just the masked")
    parser.add_argument("--link-prediction", default=False, action='store_true',
                       help="use link prediction as supervision task")
    parser.add_argument("--mask-links", default=True, action='store_true',
                       help="mask the links to be predicted")

    parser.add_argument("--batch-size", type=int, default=100,
            help="Mini-batch size. If -1, use full graph training.")
    parser.add_argument("--model_path", type=str, default=None,
            help='path for save the model')
    parser.add_argument("--fanout", type=int, default=10,
            help="Fan-out of neighbor sampling.")

    fp = parser.add_mutually_exclusive_group(required=False)
    fp.add_argument('--validation', dest='validation', action='store_true')
    fp.add_argument('--testing', dest='validation', action='store_false')
    parser.set_defaults(validation=True)
    dataset='wn18'
    args = parser.parse_args(['--dataset', dataset,'--encoder', 'RGCN'])

In [17]:
train_idx, test_idx, val_idx, labels, g, category, num_classes, masked_node_types=\
        load_data.load_hetero_data(args)

Using device cuda:0


In [18]:
mapping_dict={}
edge_lists={}
reverse_mapping_dict={}

In [19]:
g_without_features=copy.deepcopy(g)
for ntype in g_without_features.ntypes:
    del g_without_features.nodes[ntype].data['features']
homo_g=dgl.to_homo(g_without_features)
# return the mapping among id of the homogenous graph and id and ntype of the hetero graph
#not needed probabily
#mapping_dict=[(homo_g.ndata['_ID'][i],g.ntypes[int(homo_g.ndata['_TYPE'][i])]) for i in range(len(homo_g.ndata['_TYPE']))]

u,v=homo_g.all_edges()

u=u.data.numpy()
v=v.data.numpy()
u=u.astype(int)
v=v.astype(int)
edges=np.concatenate((np.expand_dims(u, axis=0),np.expand_dims(v, axis=0)),axis=0).transpose()


In [20]:
import pickle
data_folder="../data/kg/wn18/"

In [21]:
np.savetxt(data_folder+"edge_list_complete.txt",edges,fmt='%i')

In [22]:
motif_features={}

For the IMDB type of graph where a lot of relationships are among different entinties the motif representation
will be quite poor. Consider combining several relationtypes to get more interesting signals...

In [23]:
folder='../../../PGD-orbit-master/'
etype='complete'

# the row index corresponds to the original id and the value corresponds to the internal id used by nasreen
vertex_mapping_2_nasreens_ids=np.loadtxt(folder+"vertex_mapping.txt") 
mapping_from_nasreens_ids={int(vertex_mapping_2_nasreens_ids[i]): i for i in range(len(vertex_mapping_2_nasreens_ids))}
motif_per_node=np.loadtxt(folder+dataset+'_'+etype+"-node.txt",skiprows=1,delimiter=',')

for ntype in g.ntypes:
    g.nodes[ntype].data['motifs']=torch.zeros((g.number_of_nodes(ntype),motif_per_node.shape[1]-1)).int()

The following code extracts the motifs per node and then maps it as node features in the original graph by using some id mapping. It also performs some check to validate that the predicted degree is the same with the actual degree and hence the vertex mapping is correct.

In [24]:
#To check consistency see if deggrees same.
edges_dict={}
for e in edges:
    #if e[0]==3 or e[1]==3:
    #    print(e)
    e=tuple(e)
    if e not in edges_dict:
        edges_dict[e]=1
edges_li=list(edges_dict.keys())

dglonelistgraph=dgl.heterograph(data_dict={('0','1','0'):edges_li})
ntg= dgl.to_networkx(dglonelistgraph)
dgl_grp=dgl.DGLGraph(ntg)
bidercted_g=dgl.transform.to_bidirected(dgl_grp)

In [25]:

c=0
for i in range(motif_per_node.shape[0]):
    homo_id=int(mapping_from_nasreens_ids[motif_per_node[i,0]-1])
    #print('--'+str(homo_id))
    if ((bidercted_g.in_degree(homo_id)))==(int(motif_per_node[i,1])):
        c+=1
    else:
        print(bidercted_g.in_degree(homo_id))
        print(int(motif_per_node[i,1]))
    ntype=g.ntypes[homo_g.ndata['_TYPE'][homo_id]]
    ntype_id=homo_g.ndata['_ID'][homo_id]
    g.nodes[ntype].data['motifs'][ntype_id]=torch.tensor(motif_per_node[i,1:]).int()
print(c==motif_per_node.shape[0])

4
3
3
2
4
3
3
2
5
4
3
2
2
1
False


In [26]:
print(c)
print(motif_per_node.shape[0])

40936
40943


The following code retrieves the motif characteristics of each edge in the graph

In [27]:
etype='complete'
motif_per_edge=np.loadtxt(folder+dataset+'_'+etype+"-edge.txt",skiprows=1,delimiter=',')


    

Some nodes are disconnected, for this the mapping from nasreen will be -1

In [28]:
print(len(np.unique(vertex_mapping_2_nasreens_ids)))
print(len(vertex_mapping_2_nasreens_ids))
print(homo_g.number_of_nodes())
condition = (vertex_mapping_2_nasreens_ids==-1)
print(len(vertex_mapping_2_nasreens_ids[condition]))

40943
40943
40943
0


Nasreens code returns bidirectional graph by considering both directions of the edges of the original directed graphs.
Hence the returned graph has some edges that do not exist in the original graph. For now I just skip these.

The following creates a new edge, called motif that has the motif data.

In [40]:
dict_motif_edges={}
dict_motif_edata={}
for i in range(motif_per_edge.shape[0]):
    
    homo_id_dest=int(mapping_from_nasreens_ids[motif_per_edge[i,0]-1])
    homo_id_src=int(mapping_from_nasreens_ids[motif_per_edge[i,1]-1])
    #print('--'+str(homo_id))
    
    ntype_src=g.ntypes[homo_g.ndata['_TYPE'][homo_id_src]]
    ntype_id_src=homo_g.ndata['_ID'][homo_id_src]
    ntype_dest=g.ntypes[homo_g.ndata['_TYPE'][homo_id_dest]]
    ntype_id_dest=homo_g.ndata['_ID'][homo_id_dest]
    homo_e_id=homo_g.edge_id(homo_id_src,homo_id_dest)
    homo_e_id=homo_e_id.data.cpu().numpy()
    n_etype=(ntype_src,'motif_edge',ntype_dest)
    if n_etype in dict_motif_edges:
        dict_motif_edges[n_etype]+=[(int(ntype_id_src.data.cpu().numpy()),int(ntype_id_dest.data.cpu().numpy()))]
        dict_motif_edata[n_etype]+=[torch.tensor(motif_per_edge[i,2:]).int()]
    else:
        dict_motif_edges[n_etype]=[(int(ntype_id_src.data.cpu().numpy()),int(ntype_id_dest.data.cpu().numpy()))]
        dict_motif_edata[n_etype]=[torch.tensor(motif_per_edge[i,2:]).int()]

In [41]:
edge_motifs={'edata':edata,'src':src_id,'dest':dest_id}
pickle.dump(edge_motifs, open(os.path.join(data_folder, "edge_motifs.pickle"), "wb"),
                protocol=4);

In [42]:
edge_motifs

{0: {('word',
   'motif_edge',
   'word'): [tensor([    74,    516,    515,     73,     73,    515,      0,    723,    146,
            37593, 132354,   2628,   2628, 132354,      2,      1,      0,      0,
                0,      1,      0,      0,      0,      0], dtype=torch.int32), tensor([    28,    516,    515,     27,     27,    515,      0,    769,     36,
            13903, 132354,    351,    351, 132354,      2,      1,      0,      0,
                0,      1,      0,      0,      0,      0], dtype=torch.int32), tensor([  233,   306,   305,   232,   232,   305,     0,   566,  1081, 70759,
           46360, 26796, 26796, 46360,     1,     0,     0,     0,     0,     0,
               0,     0,     0,     0], dtype=torch.int32), tensor([     8,    516,    515,      7,      7,    515,      0,    771,     37,
             3585, 132354,     21,     21, 132354,     20,      1,      0,      0,
                0,      1,      0,      0,      0,      0], dtype=torch.int32), tensor([

Check that the eid for the first edge in dict_homo_edge is 1 so that the dict_homo_edata are correctly aligned

In [37]:
print(len(np.where(~dict_motif_edata[e].data.cpu().numpy()[:,:].any(axis=1))[0]))

0


Graph(num_nodes={'word': 40943},
      num_edges={('word', '12', 'word'): 34832, ('word', '17', 'word'): 34796, ('word', '16', 'word'): 7382, ('word', '8', 'word'): 29715, ('word', '9', 'word'): 2921, ('word', '13', 'word'): 1299, ('word', '5', 'word'): 7402, ('word', '0', 'word'): 3118, ('word', '6', 'word'): 4805, ('word', '10', 'word'): 2935, ('word', '15', 'word'): 3116, ('word', '1', 'word'): 4816, ('word', '14', 'word'): 629, ('word', '4', 'word'): 923, ('word', '2', 'word'): 632, ('word', '3', 'word'): 903, ('word', '7', 'word'): 1138, ('word', '11', 'word'): 80, ('word', 'motif_edge', 'word'): 75451},
      metagraph=[('word', 'word'), ('word', 'word'), ('word', 'word'), ('word', 'word'), ('word', 'word'), ('word', 'word'), ('word', 'word'), ('word', 'word'), ('word', 'word'), ('word', 'word'), ('word', 'word'), ('word', 'word'), ('word', 'word'), ('word', 'word'), ('word', 'word'), ('word', 'word'), ('word', 'word'), ('word', 'word'), ('word', 'word')])


AttributeError: can't set attribute

Here we have multiple edge ids in the hetero graph that possibly map to the same id in the graph of nasreen. The
following creates the motif edata for each of the existing links. Since the current design treats it as homogenous
focus on above.

In [100]:
result=True
for etype in g.etypes:
    g.edges[etype].data['motifs']=torch.zeros((g.number_of_edges(etype),motif_per_edge.shape[1]-2)).int()
for i in range(motif_per_edge.shape[0]):
    
    homo_id_dest=int(mapping_from_nasreens_ids[motif_per_edge[i,0]-1])
    homo_id_src=int(mapping_from_nasreens_ids[motif_per_edge[i,1]-1])
    #print('--'+str(homo_id))
    
    #print(str(homo_id_src)+','+str(homo_id_dest))
    ntype_src=g.ntypes[homo_g.ndata['_TYPE'][homo_id_src]]
    ntype_id_src=homo_g.ndata['_ID'][homo_id_src]
    ntype_dest=g.ntypes[homo_g.ndata['_TYPE'][homo_id_dest]]
    ntype_id_dest=homo_g.ndata['_ID'][homo_id_dest]
    homo_e_id=homo_g.edge_id(homo_id_src,homo_id_dest)
    homo_e_id=homo_e_id.data.cpu().numpy()

    
    if len(homo_e_id)!=0:
        homo_e_id=homo_e_id[0]
        #print('homo_id '+str(homo_e_id))
        cetype=g.etypes[homo_g.edata['_TYPE'][homo_e_id]]
        hetero_e_id=homo_g.edata['_ID'][homo_e_id]
        # TODO probably here we need to add the features for 
        # all edge types that may contain this specific src-dest pair
        
        het_e_id=g.edge_id(ntype_id_src,ntype_id_dest,etype=(ntype_src,cetype,ntype_dest))
        het_e_id=het_e_id.data.cpu().numpy()
        
        print('hetero_id '+str(het_e_id))
        #print(cetype)
        if len(het_e_id)<=1:
            het_e_id=int(het_e_id)
            result=result and (hetero_e_id==het_e_id)
            #print(result)
            g.edges[cetype].data['motifs'][het_e_id]=torch.tensor(motif_per_edge[i,2:]).int()
        else:
            # for some edge type ( participated by ) we may have multiple egdes of the same type among actor-movies.
            for eid in het_e_id:
                g.edges[cetype].data['motifs'][eid]=torch.tensor(motif_per_edge[i,2:]).int()
print(result)

hetero_id [1949]
hetero_id [1950]
hetero_id [1798]
hetero_id [1799]
hetero_id [1802]
hetero_id [1800]
hetero_id [7687]
hetero_id [1918]
hetero_id [1794]
hetero_id [1795]
hetero_id [1796]
hetero_id [1797]
hetero_id [1801]
hetero_id [1803]
hetero_id [1804]
hetero_id [1805]
hetero_id [1951]
hetero_id [1952]
hetero_id [1953]
hetero_id [1954]
hetero_id [1955]
hetero_id [1956]
hetero_id [1957]
hetero_id [1958]
hetero_id [1959]
hetero_id [7830]
hetero_id [7831]
hetero_id [7832]
hetero_id [7833]
hetero_id [7834]
hetero_id [7836]
hetero_id [7837]
hetero_id [7838]
hetero_id [7839]
hetero_id [1785]
hetero_id [1789]
hetero_id [1865]
hetero_id [7685]
hetero_id [1873]
hetero_id [1919]
hetero_id [1920]
hetero_id [1921]
hetero_id [1922]
hetero_id [1923]
hetero_id [1924]
hetero_id [1925]
hetero_id [1926]
hetero_id [1927]
hetero_id [1928]
hetero_id [1929]
hetero_id [1930]
hetero_id [2120]
hetero_id [2121]
hetero_id [2122]
hetero_id [2123]
hetero_id [2124]
hetero_id [2125]
hetero_id [2126]
hetero_id [212

hetero_id [2448]
hetero_id [2449]
hetero_id [2450]
hetero_id [2451]
hetero_id [2452]
hetero_id [2453]
hetero_id [9211]
hetero_id [9212]
hetero_id [9213]
hetero_id [9214]
hetero_id [9215]
hetero_id [9218]
hetero_id [9219]
hetero_id [9220]
hetero_id [1952]
hetero_id [2158]
hetero_id [2161]
hetero_id [2163]
hetero_id [2165]
hetero_id [8223]
hetero_id [8224]
hetero_id [8226]
hetero_id [8227]
hetero_id [8228]
hetero_id [8229]
hetero_id [8230]
hetero_id [2137]
hetero_id [2142]
hetero_id [2143]
hetero_id [8170]
hetero_id [1890]
hetero_id [1891]
hetero_id [1892]
hetero_id [1894]
hetero_id [1895]
hetero_id [2072]
hetero_id [2073]
hetero_id [2074]
hetero_id [2075]
hetero_id [2076]
hetero_id [2077]
hetero_id [2078]
hetero_id [8030]
hetero_id [8031]
hetero_id [8032]
hetero_id [8033]
hetero_id [8034]
hetero_id [8037]
hetero_id [8038]
hetero_id [8039]
hetero_id [1839]
hetero_id [1840]
hetero_id [1841]
hetero_id [1842]
hetero_id [1843]
hetero_id [2012]
hetero_id [2014]
hetero_id [2015]
hetero_id [201

hetero_id [3559]
hetero_id [3560]
hetero_id [3561]
hetero_id [3563]
hetero_id [3564]
hetero_id [3565]
hetero_id [3566]
hetero_id [534]
hetero_id [535]
hetero_id [585]
hetero_id [586]
hetero_id [587]
hetero_id [588]
hetero_id [589]
hetero_id [590]
hetero_id [591]
hetero_id [2340]
hetero_id [2341]
hetero_id [2342]
hetero_id [2345]
hetero_id [2346]
hetero_id [2347]
hetero_id [2348]
hetero_id [2349]
hetero_id [2035]
hetero_id [2036]
hetero_id [2037]
hetero_id [2044]
hetero_id [331]
hetero_id [332]
hetero_id [353]
hetero_id [354]
hetero_id [355]
hetero_id [356]
hetero_id [357]
hetero_id [358]
hetero_id [359]
hetero_id [1457]
hetero_id [1458]
hetero_id [1459]
hetero_id [1460]
hetero_id [1462]
hetero_id [1463]
hetero_id [1464]
hetero_id [1465]
hetero_id [1466]
hetero_id [1273]
hetero_id [210]
hetero_id [211]
hetero_id [212]
hetero_id [213]
hetero_id [214]
hetero_id [232]
hetero_id [233]
hetero_id [234]
hetero_id [235]
hetero_id [238]
hetero_id [239]
hetero_id [981]
hetero_id [982]
hetero_id [

hetero_id [1791]
hetero_id [1792]
hetero_id [1793]
hetero_id [7370]
hetero_id [7371]
hetero_id [7372]
hetero_id [7373]
hetero_id [7374]
hetero_id [7375]
hetero_id [7376]
hetero_id [7377]
hetero_id [7378]
hetero_id [7379]
hetero_id [6761]
hetero_id [6762]
hetero_id [1325]
hetero_id [1327]
hetero_id [1452]
hetero_id [1453]
hetero_id [1454]
hetero_id [1455]
hetero_id [5900]
hetero_id [5901]
hetero_id [5902]
hetero_id [5904]
hetero_id [5905]
hetero_id [5907]
hetero_id [5908]
hetero_id [5909]
hetero_id [5808]
hetero_id [1232]
hetero_id [1233]
hetero_id [1234]
hetero_id [1361]
hetero_id [1362]
hetero_id [1363]
hetero_id [1364]
hetero_id [5537]
hetero_id [5538]
hetero_id [5539]
hetero_id [5540]
hetero_id [5541]
hetero_id [5543]
hetero_id [5544]
hetero_id [5545]
hetero_id [5546]
hetero_id [2165]
hetero_id [5349]
hetero_id [5191]
hetero_id [5195]
hetero_id [5080]
hetero_id [1078]
hetero_id [1079]
hetero_id [1080]
hetero_id [1081]
hetero_id [1082]
hetero_id [1083]
hetero_id [1084]
hetero_id [108

hetero_id [3428]
hetero_id [3430]
hetero_id [3432]
hetero_id [3433]
hetero_id [3434]
hetero_id [3435]
hetero_id [3436]
hetero_id [3359]
hetero_id [3360]
hetero_id [717]
hetero_id [814]
hetero_id [815]
hetero_id [3248]
hetero_id [3249]
hetero_id [3250]
hetero_id [3251]
hetero_id [3252]
hetero_id [3253]
hetero_id [3254]
hetero_id [3255]
hetero_id [3257]
hetero_id [3220]
hetero_id [697]
hetero_id [786]
hetero_id [787]
hetero_id [788]
hetero_id [789]
hetero_id [790]
hetero_id [3184]
hetero_id [3185]
hetero_id [3186]
hetero_id [3187]
hetero_id [3188]
hetero_id [3189]
hetero_id [3190]
hetero_id [3191]
hetero_id [3193]
hetero_id [2259]
hetero_id [509]
hetero_id [550]
hetero_id [2244]
hetero_id [2245]
hetero_id [2246]
hetero_id [2247]
hetero_id [2248]
hetero_id [2249]
hetero_id [2250]
hetero_id [2251]
hetero_id [2252]
hetero_id [2253]
hetero_id [2286]
hetero_id [5354]
hetero_id [433]
hetero_id [434]
hetero_id [435]
hetero_id [436]
hetero_id [437]
hetero_id [482]
hetero_id [483]
hetero_id [1903

...

hetero_id [3338]
hetero_id [3339]
hetero_id [3340]
hetero_id [3341]
hetero_id [3342]
hetero_id [2785]
hetero_id [2786]
hetero_id [2787]
hetero_id [2788]
hetero_id [2789]
hetero_id [1852]
hetero_id [1853]
hetero_id [1854]
hetero_id [1855]
hetero_id [1856]
hetero_id [1857]
hetero_id [1721]
hetero_id [1449]
hetero_id [1237]
hetero_id [1238]
hetero_id [1239]
hetero_id [1240]
hetero_id [1241]
hetero_id [1242]
hetero_id [455]
hetero_id [456]
hetero_id [457]
hetero_id [458]
hetero_id [459]
hetero_id [1724]
hetero_id [1389]
hetero_id [1390]
hetero_id [5628]
hetero_id [5629]
hetero_id [5630]
hetero_id [5631]
hetero_id [947]
hetero_id [3875]
hetero_id [3876]
hetero_id [3877]
hetero_id [3878]
hetero_id [3879]
hetero_id [695]
hetero_id [696]
hetero_id [2804]
hetero_id [2805]
hetero_id [601]
hetero_id [2420]
hetero_id [2421]
hetero_id [2422]
hetero_id [2423]
hetero_id [2424]
hetero_id [776]
hetero_id [2358]
hetero_id [2688]
hetero_id [2689]
hetero_id [10202]
hetero_id [10203]
hetero_id [2347]
heter

hetero_id [227]
hetero_id [946]
hetero_id [948]
hetero_id [867]
hetero_id [869]
hetero_id [817]
hetero_id [820]
hetero_id [168]
hetero_id [690]
hetero_id [692]
hetero_id [694]
hetero_id [140]
hetero_id [141]
hetero_id [164]
hetero_id [564]
hetero_id [565]
hetero_id [566]
hetero_id [554]
hetero_id [432]
hetero_id [89]
hetero_id [61]
hetero_id [248]
hetero_id [249]
hetero_id [250]
hetero_id [251]
hetero_id [49]
hetero_id [185]
hetero_id [186]
hetero_id [187]
hetero_id [189]
hetero_id [47]
hetero_id [48]
hetero_id [49]
hetero_id [183]
hetero_id [184]
hetero_id [35]
hetero_id [38]
hetero_id [124]
hetero_id [125]
hetero_id [126]
hetero_id [127]
hetero_id [1507]
hetero_id [10004]
hetero_id [10005]
hetero_id [10006]
hetero_id [10007]
hetero_id [10008]
hetero_id [9959]
hetero_id [9960]
hetero_id [9961]
hetero_id [9962]
hetero_id [9963]
hetero_id [7537]
hetero_id [7538]
hetero_id [7539]
hetero_id [7540]
hetero_id [7541]
hetero_id [7179]
hetero_id [7180]
hetero_id [7181]
hetero_id [7182]
hetero_

hetero_id [7557]
hetero_id [7558]
hetero_id [7559]
hetero_id [7412]
hetero_id [7413]
hetero_id [7415]
hetero_id [6909]
hetero_id [6910]
hetero_id [6911]
hetero_id [6831]
hetero_id [6830]
hetero_id [5514]
hetero_id [5516]
hetero_id [5517]
hetero_id [4760]
hetero_id [4761]
hetero_id [4762]
hetero_id [4763]
hetero_id [6855]
hetero_id [6854]
hetero_id [6853]
hetero_id [4072]
hetero_id [4073]
hetero_id [4074]
hetero_id [4075]
hetero_id [3973]
hetero_id [3975]
hetero_id [8050]
hetero_id [2461]
hetero_id [2462]
hetero_id [2463]
hetero_id [2464]
hetero_id [2457]
hetero_id [2458]
hetero_id [2459]
hetero_id [2460]
hetero_id [2359]
hetero_id [2360]
hetero_id [2361]
hetero_id [2362]
hetero_id [6735]
hetero_id [1497]
hetero_id [1498]
hetero_id [1499]
hetero_id [1500]
hetero_id [1386]
hetero_id [1387]
hetero_id [1388]
hetero_id [1269]
hetero_id [1271]
hetero_id [1272]
hetero_id [1177]
hetero_id [809]
hetero_id [810]
hetero_id [811]
hetero_id [812]
hetero_id [560]
hetero_id [561]
hetero_id [562]
hete

hetero_id [4209]
hetero_id [4210]
hetero_id [4211]
hetero_id [3872]
hetero_id [3873]
hetero_id [3874]
hetero_id [3623]
hetero_id [3624]
hetero_id [3625]
hetero_id [3620]
hetero_id [3621]
hetero_id [3622]
hetero_id [2898]
hetero_id [2899]
hetero_id [2900]
hetero_id [2465]
hetero_id [2466]
hetero_id [2467]
hetero_id [1993]
hetero_id [1994]
hetero_id [1995]
hetero_id [1974]
hetero_id [1975]
hetero_id [1976]
hetero_id [1178]
hetero_id [1179]
hetero_id [910]
hetero_id [813]
hetero_id [814]
hetero_id [815]
hetero_id [786]
hetero_id [787]
hetero_id [788]
hetero_id [406]
hetero_id [407]
hetero_id [408]
hetero_id [253]
hetero_id [254]
hetero_id [255]
hetero_id [245]
hetero_id [246]
hetero_id [247]
hetero_id [2233]
hetero_id [8442]
hetero_id [8443]
hetero_id [1679]
hetero_id [6975]
hetero_id [6976]
hetero_id [1289]
hetero_id [5223]
hetero_id [5224]
hetero_id [1958]
hetero_id [1959]
hetero_id [2697]
hetero_id [10223]
hetero_id [2363]
hetero_id [2364]
hetero_id [10216]
hetero_id [10214]
hetero_id 

hetero_id [5474]
hetero_id [1208]
hetero_id [1319]
hetero_id [1191]
hetero_id [1246]
hetero_id [1116]
hetero_id [4883]
hetero_id [3013]
hetero_id [3012]
hetero_id [1108]
hetero_id [1213]
hetero_id [1105]
hetero_id [1106]
hetero_id [1089]
hetero_id [4777]
hetero_id [1086]
hetero_id [1087]
hetero_id [1067]
hetero_id [1066]
hetero_id [1175]
hetero_id [1053]
hetero_id [4653]
hetero_id [1052]
hetero_id [4649]
hetero_id [1048]
hetero_id [1049]
hetero_id [1044]
hetero_id [4627]
hetero_id [1038]
hetero_id [4591]
hetero_id [1036]
hetero_id [1139]
hetero_id [993]
hetero_id [4384]
hetero_id [984]
hetero_id [4350]
hetero_id [983]
hetero_id [1072]
hetero_id [975]
hetero_id [4302]
hetero_id [963]
hetero_id [1049]
hetero_id [961]
hetero_id [4245]
hetero_id [4207]
hetero_id [950]
hetero_id [4181]
hetero_id [921]
hetero_id [922]
hetero_id [918]
hetero_id [4071]
hetero_id [917]
hetero_id [4063]
hetero_id [876]
hetero_id [3858]
hetero_id [849]
hetero_id [3789]
hetero_id [818]
hetero_id [901]
hetero_id [8

SOS The same edge that corresponds to different edge types is counted multiple timed by the dgl.graph.in_degrees implementation. On the other hand multiple edges are ignored in Nasreens code. This may lead to a discrepancy in the degree reported by her code and dgl. 

In [97]:
#(torch.sum(,1))
etype='written_by'
print(len(g.edges[etype].data['motifs']))
print(len(np.where(~g.edges[etype].data['motifs'].data.cpu().numpy().any(axis=1))[0]))
sum(g.edges[etype].data['motifs'].data.cpu().numpy())

2704
642


array([  2240,  28581,  26519,    178,    178,  26519,      0,   3264,
         2186,   2258, 206296,     38,     38, 206296,    216,      0,
            0,      0,      0,      0,      0,      0,      0,      0],
      dtype=int32)

In [98]:

pickle.dump(g, open(os.path.join(data_folder, "graph_reduced_m.pickle"), "wb"),
                protocol=4);

In [8]:
import pickle
data_folder="../data/kg/wn18/"

In [11]:
g=pickle.load(open(os.path.join(data_folder, "graph_reduced_m.pickle"), "rb"))

In [14]:
g.edges['12'].data

{}